In [1]:
import time
import random
import threading

In [44]:
class Bar():
    # Estrutura do bar, contendo clientes e garçons - além da capacidade de cada garçom, 
    # número de rodadas grátis, estruturas para gerenciamento de pedidos (lista_de_pedidos e lista_de_clientes_querendo_pedir)
    # além de semáforos para ordenar a escrita e remoção nas estruturas de gerenciamento citadas
    def __init__(self, clientes, garcons, capacidade_garcons, rodadas):
        self.lista_clientes = [Cliente(self) for c in range(clientes)]
        self.lista_garcons = [Garcom(self) for g in range(garcons)]
        self.capacidade_garcons = capacidade_garcons
        self.rodadas_gratis = rodadas
        
        self.lista_de_pedidos = self.getListaDePedidos()
        self.lista_de_pedidos_sendo_consumidos = list()
        self.lista_de_clientes_querendo_pedir = list()
        
    
    def rodadaAberta(self):
        return self.getQuantidadeClientesQuerendoPedir() > 0
     
        
    def getListaDePedidos(self):
        return {g: [] for g in self.lista_garcons}

    
    def getRodadasGratis(self):
        return self.rodadas_gratis
        
        
    def getQuantidadeClientesQuerendoPedir(self):  
        # Tamanho da lista de clientes que querem fazer pedido
        return len(self.lista_de_clientes_querendo_pedir)
    
    
    def getClienteQuerendoPedir(self):
        # Retorna primeiro cliente da lista de clientes que desejam fazer pedido
        return self.lista_de_clientes_querendo_pedir.pop(0)
      
        
    def requererPedido(self, cliente):
        # Lista com clientes que querem fazer pedido naquela rodada
        cliente.sem.acquire()
        
        self.lista_de_clientes_querendo_pedir.append(cliente)
        
        cliente.sem.release()
        
            
    def registraPedido(self, garcom):
        # Garçom - passado por parâmetro - registra pedido do primeiro cliente na lista
        # Existe um semáforo a fim de que esse garçom termine o resgistro antes que outro faça outro pedido
        # Essa abordagem foi pensada para não haver conflito de escrita - causando sobreposição na lista
        garcom.sem.acquire()
        
        cliente = self.getClienteQuerendoPedir()
        self.lista_de_pedidos[garcom] += [cliente]
        cliente.esperaPedido()
        print('O garçom ' + str(garcom) + ' registrou o pedido do cliente ' + str(cliente), flush=True)
        
        garcom.sem.release()
    
    
    def getProximoCliente(self, garcom):
        cliente = self.lista_de_pedidos[garcom].pop(0) if len(self.lista_de_pedidos[garcom]) > 0 else None
        return cliente
    
    
    def inserePedidosSendoConsumidos(self, pedido):
        self.lista_de_pedidos_sendo_consumidos.append(pedido)
        
        
    def rodada(self):
        # Função que chama os objetos thread para executar os pedidos. 
        for cliente in self.lista_clientes:
            cliente.run()
        
        print('\n\n')
        garcom = 0 
        while bar.rodadaAberta(): # todos clientes que desejam bebida devem 
            self.lista_garcons[garcom].run()                 # ser atendidos para outra rodada ser aberta
            if garcom < (len(self.lista_garcons) - 1):
                garcom += 1
            else:
                garcom = 0
            
        for pedido in self.lista_de_pedidos_sendo_consumidos:
            pedido.join()
            
        self.lista_de_pedidos_sendo_consumidos = list()
            
    def comecarRodadasGratis(self):
        # Função com a execução de todas rodadas
        for r in range(self.rodadas_gratis):
            print('COMEÇANDO RODADA ' + str(r + 1) + '/' + str(self.rodadas_gratis) + '\n\n', flush=True)
            self.rodada()
            print('\n\n\n', flush=True)

In [45]:
class Cliente(threading.Thread): 
    # Estrutura do cliente, como objeto thread 
    # Possui booleanos de validação em que estágio do pedido o cliente está 
    # Além disso, possui o objeto bar em que está conectado - para interagir com o bar e garçons
    def __init__(self, bar):
        threading.Thread.__init__(self)
        
        self.bar = bar # objeto do bar em que está

        self.sem = threading.Semaphore(1)
        
        self.fazer_pedido = True 
        self.esperando_pedido = False
        self.consumindo_pedido = False
        self.atendido = False
    
    
    def fazerPedido(self):
        # Função que decide se o cliente irá fazer pedido essa rodada, 
        # Caso o cliente vá fazer pedido, ele irá requerer seu pedido ao bar - utilizando a função requererPedido
        self.fazer_pedido = random.choice([True, True, False]) 
        if self.fazer_pedido:
            print('O cliente ' + str(self) + ' deseja fazer um pedido essa rodada.', flush=True)
            self.bar.requererPedido(self)
        else: 
            print('O cliente ' + str(self) + ' NÃO deseja fazer um pedido essa rodada.', flush=True)
        
        
    def esperaPedido(self):
        # Função para atualizar o estado do cliente como esperando seu pedido
        self.fazer_pedido = False
        self.esperando_pedido = True
        self.atendido = True
        
        
    def consomePedido(self):
        # Função que representa o cliente consumindo sua bebida depois de um tempo aleatório
        self.esperando_pedido = False
        tempo_de_consumo = random.randint(1,10) # segundos aleatórios para consumir a bebida
        self.consumindo_pedido = True
        print('O cliente ' + str(self) + ' irá consumir seu pedido em ' + str(tempo_de_consumo) + ' segundos\n', flush=True)
        time.sleep(tempo_de_consumo)
        self.consumindo_pedido = False
            
            
    def run(self):
        # Função sobrescrita da classe thread, onde o que é executado é executado em uma thread
        self.atendido = False
        if bar.getRodadasGratis() > 0 and (not self.consumindo_pedido and not self.esperando_pedido): 
            self.fazerPedido()

In [46]:
class Garcom(threading.Thread): 
    # Estrutura do cliente, como objeto thread
    # Possui um inteiro a fim de contabilizar os pedidos recebidos - para não receber mais pedidos que 
    # sua capacidade por vez
    # Além disso, possui o objeto bar em que está conectado - para interagir com o bar e clientes
    def __init__(self, bar):
        threading.Thread.__init__(self)
        
        self.bar = bar # objeto do bar em que está
        self.pedidos_recebidos = 0

        self.sem = threading.Semaphore(1)        
        
    def recebePedidos(self):   
        # Função que recebe perdido e interage com o bar
        self.bar.registraPedido(self)
        self.pedidos_recebidos += 1
        
        
    def entregaPedidos(self):
        # Função que entrega o pedido e interage com o bar
        
        for c in range(self.bar.capacidade_garcons):
            cliente = self.bar.getProximoCliente(self) 
            
            if cliente != None:
                pedido = threading.Thread(target=cliente.consomePedido)
                pedido.start()

                self.pedidos_recebidos -= 1
                self.bar.inserePedidosSendoConsumidos(pedido)
        
    def run(self):
        # Função sobrescrita da classe thread, onde o que é executado é executado em uma thread
        while self.pedidos_recebidos < self.bar.capacidade_garcons and self.bar.rodadaAberta():
            self.recebePedidos()
            
        if self.pedidos_recebidos == self.bar.capacidade_garcons or self.bar.getQuantidadeClientesQuerendoPedir() == 0:
            while self.pedidos_recebidos > 0:
                self.entregaPedidos()

In [47]:
bar = Bar(10, 2, 2, 2)

In [49]:
bar.comecarRodadasGratis()

COMEÇANDO RODADA 1/2


O cliente <Cliente(Thread-244, initial)> NÃO deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-245, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-246, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-247, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-248, initial)> NÃO deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-249, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-250, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-251, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-252, initial)> deseja fazer um pedido essa rodada.
O cliente <Cliente(Thread-253, initial)> deseja fazer um pedido essa rodada.



O garçom <Garcom(Thread-254, initial)> registrou o pedido do cliente <Cliente(Thread-245, initial)>
O garçom <Garcom(Thread-254, initial)> registrou o pedido do cliente <Cliente(Thread-246, initia